# Pix2Pix

**기억할 점**
- **한 도메인에 있는 이미지를 다른 도메인으로 해석해보자**는 관점
- Image to Image Mapping Network에서 Photo-realistic을 추구하고자 함
- GAN의 adverserial을 사용
- U-Net과 PatchGAN 등을 이용해서 성능을 최적화함
- 양쪽 도메인에 대응되는 데이터 쌍이 존재해야 함(반면 cyclegan은 데이터 쌍이 없어도 됨)

## Pix2Pix의 손실함수
- Adverserial Loss

$$L_{cGAN}(G, D) = E_{y\sim p_{data}(y)} \big[\text{log}D(y) \big] + E_{x\sim P_{data}(x), z \sim P_{z}(z)} \big[\| y - G(x, z) \|_{1}\big]$$


- Reconstruction Loss

$$L_{L1}\big(G \big) = E_{x, y \sim P_{data}(x, y), z \sim p_{z}(z)} \big[ \| y - G(x, z) \|_{1} \big]$$

- Total Loss

$$G^{*} = arg \underset{G}{\operatorname{min}} \underset{D}{\operatorname{max}} L_{cGAN}(G, D) + \lambda L_{L1}(G)$$

## 1. Import Libs

In [1]:
from __future__ import print_function
import argparse
import os
from math import log10

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from networks import define_G, define_D, GANLoss, print_network
from data import get_training_set, get_test_set
import torch.backends.cudnn as cudnn

## 2. Setting Hyperparameters

In [2]:
parser = argparse.ArgumentParser(description='pix2pix-PyTorch-implementation')
parser.add_argument('--dataset', required=False, default='facades',help='facades')
parser.add_argument('--batchSize', type=int, default=1, help='training batch size')
parser.add_argument('--testBatchSize', type=int, default=1, help='testing batch size')
parser.add_argument('--nEpochs', type=int, default=200, help='number of epochs to train for')
parser.add_argument('--input_nc', type=int, default=3, help='input image channels')
parser.add_argument('--output_nc', type=int, default=3, help='output image channels')
parser.add_argument('--ngf', type=int, default=64, help='generator filters in first conv layer')
parser.add_argument('--ndf', type=int, default=64, help='discriminator filters in first conv layer')
parser.add_argument('--lr', type=float, default=0.0002, help='Learning Rate. Default=0.002')
parser.add_argument('--beta1', type=float, default=0.5, help='beta1 for adam. default=0.5')
parser.add_argument('--cuda', action='store_true', default=True, help='use cuda?')
parser.add_argument('--threads', type=int, default=4, help='number of threads for data loader to use')
parser.add_argument('--seed', type=int, default=123, help='random seed to use. Default=123')
parser.add_argument('--lamb', type=int, default=10, help='weight on L1 term in objective')
params = parser.parse_args([])

print(params)

if params.cuda and not torch.cuda.is_available():
    raise Exception("No GPU found, please run without --cuda")

cudnn.benchmark = True

torch.manual_seed(params.seed)
if params.cuda:
    torch.cuda.manual_seed(params.seed)

Namespace(batchSize=1, beta1=0.5, cuda=True, dataset='facades', input_nc=3, lamb=10, lr=0.0002, nEpochs=200, ndf=64, ngf=64, output_nc=3, seed=123, testBatchSize=1, threads=4)


## 3. Load Dataset

In [3]:
root_path = 'dataset/'
train_set = get_training_set(root_path + params.dataset)
test_set = get_test_set(root_path + params.dataset)

train_data_loader = DataLoader(dataset=train_set, num_workers=params.threads, batch_size=params.batchSize, shuffle=True)
test_data_loader = DataLoader(dataset=test_set, num_workers=params.threads, batch_size=params.batchSize, shuffle=False)

## 4. Build Model & Optimizers & Criterions
### 4.1 Build Model

In [4]:
netG = define_G(params.input_nc, params.output_nc, params.ngf, 'batch', False, [0])
netD = define_D(params.input_nc + params.output_nc, params.ndf, 'batch', False, [0])

print(netG)
print(netD)

ResnetGenerator(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU(inplace)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (5): ReLU(inplace)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (8): ReLU(inplace)
    (9): ResnetBlock(
      (conv_block): Sequential(
        (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
        (2): ReLU(inplace)
        (3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      )
    )
    (10): ResnetBlock(
      (conv_block): Sequential(
        (0): Conv2d(256, 

### 4.2 Optimizers

In [5]:
optimizerG = optim.Adam(netG.parameters(), lr=params.lr, betas=(params.beta1, 0.999))
optimizerD = optim.Adam(netD.parameters(), lr=params.lr, betas=(params.beta1, 0.999))

### 4.3 Loss Functions

In [6]:
criterionGAN = GANLoss()
criterionL1 = nn.L1Loss()
criterionMSE = nn.MSELoss()

real_a = torch.FloatTensor(params.batchSize, params.input_nc, 256, 256)
real_b = torch.FloatTensor(params.batchSize, params.output_nc, 256, 256)

if params.cuda:
    netD = netD.cuda()
    netG = netG.cuda()
    criterionGAN = criterionGAN.cuda()
    criterionL1 = criterionL1.cuda()
    criterionMSE = criterionMSE.cuda()
    real_a = real_a.cuda()
    real_b = real_b.cuda()

real_a = Variable(real_a)
real_b = Variable(real_b)

## 5. Training Models

In [7]:
for epoch in range(1, params.nEpochs + 1):
    for iteration, batch in enumerate(train_data_loader, 1):
        # forward
        real_a_cpu, real_b_cpu = batch[0], batch[1]
        real_a.data.resize_(real_a_cpu.size()).copy_(real_a_cpu)
        real_b.data.resize_(real_b_cpu.size()).copy_(real_b_cpu)
        
        fake_b = netG(real_a)
        
        ## Discriminator network : max log(D(x, y)) + log(1 - D(x, G(x)))
        optimizerD.zero_grad()
        
        # train fake
        fake_ab = torch.cat((real_a, fake_b), 1)
        pred_fake = netD.forward(fake_ab.detach())
        loss_d_fake = criterionGAN(pred_fake, False)
        
        # train real
        real_ab = torch.cat((real_a, real_b), 1)
        pred_real = netD.forward(real_ab)
        loss_d_real = criterionGAN(pred_real, True)
        
        # combine loss
        loss_d = (loss_d_fake + loss_d_real) * 0.5
        
        loss_d.backward()
        
        optimizerD.step()
        
        ## Generator network : max log(D(x, G(x))) + L1(y, G(x))
        optimizerG.zero_grad()
        
        # 1. G(A) should fake the discriminator
        fake_ab = torch.cat((real_a, fake_b), 1)
        pred_fake = netD.forward(fake_ab)
        loss_g_gan = criterionGAN(pred_fake, True)
        
        # 2. G(A) = B
        loss_g_l1 = criterionL1(fake_b, real_b) * params.lamb
        loss_g = loss_g_gan + loss_g_l1
        loss_g.backward()
        optimizerG.step()
        
        print("===> Epoch[{}]({}/{}): Loss_D: {:.4f} Loss_G: {:.4f}".format(
            epoch, iteration, len(train_data_loader), loss_d.data[0], loss_g.data[0]))
    
    avg_psnr = 0
    for batch in test_data_loader:
        input2, target = Variable(batch[0], volatile=True), Variable(batch[1], volatile=True)
        if params.cuda:
            input2 = input2.cuda()
            target = target.cuda()

        prediction = netG(input2)
        mse = criterionMSE(prediction, target)
        psnr = 10 * log10(1 / mse.data[0])
        avg_psnr += psnr
        print("===> Avg. PSNR: {:.4f} dB".format(avg_psnr / len(test_data_loader)))

    if epoch % 50 == 0:
        if not os.path.exists('save_model'):
            os.mkdir('save_model')
        if not os.path.exists(os.path.join('save_model', params.dataset)):
            os.mkdir(os.path.join('save_model', params.dataset))
        net_g_model_out_path = "save_model/{}/netG_model_epoch_{}.pkl".format(params.dataset, epoch)
        net_d_model_out_path = 'save_model/{}/netD_model_epoch_{}.pkl'.format(params.dataset, epoch)
        torch.save(netG, net_g_model_out_path)
        torch.save(netD, net_d_model_out_path)
        print("model saved to {}".format("model " + params.dataset))

===> Epoch[1](1/200): Loss_D: 1.8677 Loss_G: 19.0809
===> Epoch[1](2/200): Loss_D: 3.9672 Loss_G: 11.6901
===> Epoch[1](3/200): Loss_D: 2.6368 Loss_G: 9.5465
===> Epoch[1](4/200): Loss_D: 2.8620 Loss_G: 9.1093
===> Epoch[1](5/200): Loss_D: 1.9276 Loss_G: 7.4764
===> Epoch[1](6/200): Loss_D: 1.5495 Loss_G: 6.9021
===> Epoch[1](7/200): Loss_D: 1.6840 Loss_G: 7.6004
===> Epoch[1](8/200): Loss_D: 1.3741 Loss_G: 7.2957
===> Epoch[1](9/200): Loss_D: 1.3231 Loss_G: 7.2742
===> Epoch[1](10/200): Loss_D: 1.9614 Loss_G: 7.4481
===> Epoch[1](11/200): Loss_D: 1.0777 Loss_G: 6.0928
===> Epoch[1](12/200): Loss_D: 1.6399 Loss_G: 7.1503
===> Epoch[1](13/200): Loss_D: 1.2928 Loss_G: 6.4189
===> Epoch[1](14/200): Loss_D: 1.5039 Loss_G: 5.4244
===> Epoch[1](15/200): Loss_D: 1.2950 Loss_G: 6.5970
===> Epoch[1](16/200): Loss_D: 1.0484 Loss_G: 6.3736
===> Epoch[1](17/200): Loss_D: 0.9101 Loss_G: 6.0247
===> Epoch[1](18/200): Loss_D: 0.8367 Loss_G: 5.5239
===> Epoch[1](19/200): Loss_D: 0.8426 Loss_G: 6.2045


===> Epoch[1](156/200): Loss_D: 0.3065 Loss_G: 4.6845
===> Epoch[1](157/200): Loss_D: 0.4805 Loss_G: 3.3222
===> Epoch[1](158/200): Loss_D: 0.2741 Loss_G: 4.9702
===> Epoch[1](159/200): Loss_D: 0.3308 Loss_G: 4.5240
===> Epoch[1](160/200): Loss_D: 0.3167 Loss_G: 4.4888
===> Epoch[1](161/200): Loss_D: 0.2513 Loss_G: 4.5292
===> Epoch[1](162/200): Loss_D: 0.2783 Loss_G: 3.0251
===> Epoch[1](163/200): Loss_D: 0.2626 Loss_G: 3.3439
===> Epoch[1](164/200): Loss_D: 0.3139 Loss_G: 2.7899
===> Epoch[1](165/200): Loss_D: 0.2759 Loss_G: 4.4711
===> Epoch[1](166/200): Loss_D: 0.3374 Loss_G: 3.5718
===> Epoch[1](167/200): Loss_D: 0.2643 Loss_G: 5.0037
===> Epoch[1](168/200): Loss_D: 0.3219 Loss_G: 3.9325
===> Epoch[1](169/200): Loss_D: 0.3348 Loss_G: 3.6637
===> Epoch[1](170/200): Loss_D: 0.3236 Loss_G: 3.1912
===> Epoch[1](171/200): Loss_D: 0.3209 Loss_G: 3.3690
===> Epoch[1](172/200): Loss_D: 0.3598 Loss_G: 3.4131
===> Epoch[1](173/200): Loss_D: 0.3182 Loss_G: 4.9204
===> Epoch[1](174/200): Loss

===> Epoch[2](25/200): Loss_D: 0.3399 Loss_G: 4.4527
===> Epoch[2](26/200): Loss_D: 0.3076 Loss_G: 4.0271
===> Epoch[2](27/200): Loss_D: 0.3056 Loss_G: 4.0375
===> Epoch[2](28/200): Loss_D: 0.2770 Loss_G: 4.2932
===> Epoch[2](29/200): Loss_D: 0.2901 Loss_G: 3.2827
===> Epoch[2](30/200): Loss_D: 0.2510 Loss_G: 3.2967
===> Epoch[2](31/200): Loss_D: 0.3319 Loss_G: 3.5702
===> Epoch[2](32/200): Loss_D: 0.3935 Loss_G: 3.5054
===> Epoch[2](33/200): Loss_D: 0.2264 Loss_G: 4.6333
===> Epoch[2](34/200): Loss_D: 0.3040 Loss_G: 3.4845
===> Epoch[2](35/200): Loss_D: 0.2566 Loss_G: 4.4287
===> Epoch[2](36/200): Loss_D: 0.2565 Loss_G: 4.0961
===> Epoch[2](37/200): Loss_D: 0.2513 Loss_G: 4.2095
===> Epoch[2](38/200): Loss_D: 0.3498 Loss_G: 4.5951
===> Epoch[2](39/200): Loss_D: 0.3037 Loss_G: 3.5901
===> Epoch[2](40/200): Loss_D: 0.3761 Loss_G: 5.2543
===> Epoch[2](41/200): Loss_D: 0.2594 Loss_G: 3.1778
===> Epoch[2](42/200): Loss_D: 0.3364 Loss_G: 4.4613
===> Epoch[2](43/200): Loss_D: 0.3283 Loss_G: 

===> Epoch[2](179/200): Loss_D: 0.2142 Loss_G: 3.6353
===> Epoch[2](180/200): Loss_D: 0.1900 Loss_G: 3.0309
===> Epoch[2](181/200): Loss_D: 0.1984 Loss_G: 5.2674
===> Epoch[2](182/200): Loss_D: 0.2585 Loss_G: 3.1697
===> Epoch[2](183/200): Loss_D: 0.2157 Loss_G: 3.4144
===> Epoch[2](184/200): Loss_D: 0.1289 Loss_G: 3.8454
===> Epoch[2](185/200): Loss_D: 0.2457 Loss_G: 4.0103
===> Epoch[2](186/200): Loss_D: 0.3045 Loss_G: 2.8039
===> Epoch[2](187/200): Loss_D: 0.2885 Loss_G: 3.6409
===> Epoch[2](188/200): Loss_D: 0.1951 Loss_G: 4.0713
===> Epoch[2](189/200): Loss_D: 0.1525 Loss_G: 3.1765
===> Epoch[2](190/200): Loss_D: 0.1158 Loss_G: 5.1474
===> Epoch[2](191/200): Loss_D: 0.3121 Loss_G: 3.3087
===> Epoch[2](192/200): Loss_D: 0.2091 Loss_G: 3.5058
===> Epoch[2](193/200): Loss_D: 0.2711 Loss_G: 4.1502
===> Epoch[2](194/200): Loss_D: 0.2462 Loss_G: 3.5963
===> Epoch[2](195/200): Loss_D: 0.3249 Loss_G: 3.2670
===> Epoch[2](196/200): Loss_D: 0.3582 Loss_G: 4.5636
===> Epoch[2](197/200): Loss

===> Epoch[3](48/200): Loss_D: 0.3756 Loss_G: 3.7921
===> Epoch[3](49/200): Loss_D: 0.1970 Loss_G: 3.5223
===> Epoch[3](50/200): Loss_D: 0.2262 Loss_G: 5.3818
===> Epoch[3](51/200): Loss_D: 0.1433 Loss_G: 4.5283
===> Epoch[3](52/200): Loss_D: 0.1845 Loss_G: 3.4217
===> Epoch[3](53/200): Loss_D: 0.1988 Loss_G: 4.2446
===> Epoch[3](54/200): Loss_D: 0.1684 Loss_G: 4.2350
===> Epoch[3](55/200): Loss_D: 0.2919 Loss_G: 4.0404
===> Epoch[3](56/200): Loss_D: 0.2200 Loss_G: 3.1637
===> Epoch[3](57/200): Loss_D: 0.3583 Loss_G: 3.6329
===> Epoch[3](58/200): Loss_D: 0.2314 Loss_G: 4.9912
===> Epoch[3](59/200): Loss_D: 0.2330 Loss_G: 4.9412
===> Epoch[3](60/200): Loss_D: 0.2943 Loss_G: 3.0807
===> Epoch[3](61/200): Loss_D: 0.3127 Loss_G: 3.2194
===> Epoch[3](62/200): Loss_D: 0.1956 Loss_G: 4.4659
===> Epoch[3](63/200): Loss_D: 0.2473 Loss_G: 3.0844
===> Epoch[3](64/200): Loss_D: 0.1723 Loss_G: 4.7496
===> Epoch[3](65/200): Loss_D: 0.1565 Loss_G: 4.2899
===> Epoch[3](66/200): Loss_D: 0.2438 Loss_G: 

...

===> Epoch[198](160/200): Loss_D: 0.1172 Loss_G: 2.3130
===> Epoch[198](161/200): Loss_D: 0.0544 Loss_G: 2.5468
===> Epoch[198](162/200): Loss_D: 0.1204 Loss_G: 2.1924
===> Epoch[198](163/200): Loss_D: 0.0477 Loss_G: 1.8401
===> Epoch[198](164/200): Loss_D: 0.0707 Loss_G: 2.1241
===> Epoch[198](165/200): Loss_D: 0.0461 Loss_G: 2.0227
===> Epoch[198](166/200): Loss_D: 0.0862 Loss_G: 1.6749
===> Epoch[198](167/200): Loss_D: 0.1101 Loss_G: 1.8033
===> Epoch[198](168/200): Loss_D: 0.0742 Loss_G: 1.8967
===> Epoch[198](169/200): Loss_D: 0.0914 Loss_G: 2.0359
===> Epoch[198](170/200): Loss_D: 0.0479 Loss_G: 2.7888
===> Epoch[198](171/200): Loss_D: 0.0814 Loss_G: 1.7620
===> Epoch[198](172/200): Loss_D: 0.0624 Loss_G: 2.0564
===> Epoch[198](173/200): Loss_D: 0.0580 Loss_G: 2.1799
===> Epoch[198](174/200): Loss_D: 0.0748 Loss_G: 2.2678
===> Epoch[198](175/200): Loss_D: 0.0554 Loss_G: 2.0743
===> Epoch[198](176/200): Loss_D: 0.0443 Loss_G: 2.2184
===> Epoch[198](177/200): Loss_D: 0.0694 Loss_G:

===> Epoch[199](25/200): Loss_D: 0.0602 Loss_G: 2.1862
===> Epoch[199](26/200): Loss_D: 0.0745 Loss_G: 2.0918
===> Epoch[199](27/200): Loss_D: 0.0557 Loss_G: 1.8523
===> Epoch[199](28/200): Loss_D: 0.0746 Loss_G: 2.2327
===> Epoch[199](29/200): Loss_D: 0.0920 Loss_G: 2.0847
===> Epoch[199](30/200): Loss_D: 0.0893 Loss_G: 1.7751
===> Epoch[199](31/200): Loss_D: 0.1195 Loss_G: 2.4678
===> Epoch[199](32/200): Loss_D: 0.1104 Loss_G: 1.9393
===> Epoch[199](33/200): Loss_D: 0.0724 Loss_G: 1.9459
===> Epoch[199](34/200): Loss_D: 0.0531 Loss_G: 2.2709
===> Epoch[199](35/200): Loss_D: 0.0448 Loss_G: 2.0245
===> Epoch[199](36/200): Loss_D: 0.0676 Loss_G: 2.3661
===> Epoch[199](37/200): Loss_D: 0.0942 Loss_G: 2.1217
===> Epoch[199](38/200): Loss_D: 0.1365 Loss_G: 2.3899
===> Epoch[199](39/200): Loss_D: 0.0488 Loss_G: 2.1999
===> Epoch[199](40/200): Loss_D: 0.1072 Loss_G: 2.4201
===> Epoch[199](41/200): Loss_D: 0.1302 Loss_G: 1.6073
===> Epoch[199](42/200): Loss_D: 0.2293 Loss_G: 2.5860
===> Epoch

===> Epoch[199](175/200): Loss_D: 0.0840 Loss_G: 2.2871
===> Epoch[199](176/200): Loss_D: 0.0396 Loss_G: 2.2490
===> Epoch[199](177/200): Loss_D: 0.0619 Loss_G: 2.1255
===> Epoch[199](178/200): Loss_D: 0.0605 Loss_G: 1.9697
===> Epoch[199](179/200): Loss_D: 0.0591 Loss_G: 1.7491
===> Epoch[199](180/200): Loss_D: 0.1265 Loss_G: 2.1945
===> Epoch[199](181/200): Loss_D: 0.0999 Loss_G: 2.0738
===> Epoch[199](182/200): Loss_D: 0.1012 Loss_G: 1.9123
===> Epoch[199](183/200): Loss_D: 0.0516 Loss_G: 1.6952
===> Epoch[199](184/200): Loss_D: 0.0917 Loss_G: 2.6296
===> Epoch[199](185/200): Loss_D: 0.0706 Loss_G: 2.1955
===> Epoch[199](186/200): Loss_D: 0.0464 Loss_G: 1.8769
===> Epoch[199](187/200): Loss_D: 0.0951 Loss_G: 2.6311
===> Epoch[199](188/200): Loss_D: 0.0565 Loss_G: 1.9226
===> Epoch[199](189/200): Loss_D: 0.0638 Loss_G: 2.2280
===> Epoch[199](190/200): Loss_D: 0.0814 Loss_G: 2.1729
===> Epoch[199](191/200): Loss_D: 0.0413 Loss_G: 1.9781
===> Epoch[199](192/200): Loss_D: 0.0373 Loss_G:

===> Epoch[200](40/200): Loss_D: 0.0610 Loss_G: 1.9206
===> Epoch[200](41/200): Loss_D: 0.0526 Loss_G: 2.2431
===> Epoch[200](42/200): Loss_D: 0.0454 Loss_G: 1.9259
===> Epoch[200](43/200): Loss_D: 0.0538 Loss_G: 2.5450
===> Epoch[200](44/200): Loss_D: 0.0458 Loss_G: 2.0281
===> Epoch[200](45/200): Loss_D: 0.0621 Loss_G: 1.8258
===> Epoch[200](46/200): Loss_D: 0.0495 Loss_G: 2.1825
===> Epoch[200](47/200): Loss_D: 0.0492 Loss_G: 2.0491
===> Epoch[200](48/200): Loss_D: 0.0707 Loss_G: 2.2347
===> Epoch[200](49/200): Loss_D: 0.0310 Loss_G: 1.9741
===> Epoch[200](50/200): Loss_D: 0.0637 Loss_G: 1.6752
===> Epoch[200](51/200): Loss_D: 0.0807 Loss_G: 2.4090
===> Epoch[200](52/200): Loss_D: 0.0716 Loss_G: 2.1105
===> Epoch[200](53/200): Loss_D: 0.0379 Loss_G: 2.2418
===> Epoch[200](54/200): Loss_D: 0.0679 Loss_G: 1.8516
===> Epoch[200](55/200): Loss_D: 0.0554 Loss_G: 1.7597
===> Epoch[200](56/200): Loss_D: 0.0560 Loss_G: 2.3562
===> Epoch[200](57/200): Loss_D: 0.0271 Loss_G: 2.2385
===> Epoch

===> Epoch[200](190/200): Loss_D: 0.0986 Loss_G: 1.9354
===> Epoch[200](191/200): Loss_D: 0.0699 Loss_G: 2.1442
===> Epoch[200](192/200): Loss_D: 0.0485 Loss_G: 2.1079
===> Epoch[200](193/200): Loss_D: 0.0634 Loss_G: 2.1080
===> Epoch[200](194/200): Loss_D: 0.0808 Loss_G: 2.4024
===> Epoch[200](195/200): Loss_D: 0.0693 Loss_G: 2.7309
===> Epoch[200](196/200): Loss_D: 0.1374 Loss_G: 1.6458
===> Epoch[200](197/200): Loss_D: 0.0596 Loss_G: 1.6739
===> Epoch[200](198/200): Loss_D: 0.0686 Loss_G: 2.1723
===> Epoch[200](199/200): Loss_D: 0.0852 Loss_G: 3.0075
===> Epoch[200](200/200): Loss_D: 0.0678 Loss_G: 2.3021
===> Avg. PSNR: 0.0529 dB
===> Avg. PSNR: 0.0972 dB
===> Avg. PSNR: 0.1208 dB
===> Avg. PSNR: 0.1556 dB
===> Avg. PSNR: 0.1882 dB
===> Avg. PSNR: 0.2254 dB
===> Avg. PSNR: 0.2535 dB
===> Avg. PSNR: 0.2903 dB
===> Avg. PSNR: 0.3280 dB
===> Avg. PSNR: 0.3660 dB
===> Avg. PSNR: 0.4095 dB
===> Avg. PSNR: 0.4651 dB
===> Avg. PSNR: 0.4916 dB
===> Avg. PSNR: 0.5309 dB
===> Avg. PSNR: 0.58